In [3]:
import numpy as np
import pandas as pd
import pickle as pk
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from datetime import datetime, date
import calendar
from sklearn import ensemble
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Loading the dataset 
currentdf  = pd.read_csv(r"C:\Users\Russell\Desktop\SmartParkingApp\PL80_Dataset_1.csv")
currentdf = currentdf.dropna()
currentdf = currentdf.sample(frac=1)

print(currentdf.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11548 entries, 4404 to 8487
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   SESSIONID         11548 non-null  object 
 1   CARPARKNUMBER     11548 non-null  object 
 2   VEHICLETYPE       11548 non-null  object 
 3   SESSIONSTART      11548 non-null  object 
 4   SESSIONEND        11548 non-null  object 
 5   TOTAL_CHARGE      11548 non-null  float64
 6   DURATION          11548 non-null  int64  
 7   EFFECTIVE_CHARGE  11548 non-null  float64
 8   LOT_NUMBER        11548 non-null  int64  
 9   LABEL_TYPE        11548 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 992.4+ KB
None


In [5]:
#Checking the current Data Frame
currentdf

,SESSIONID,CARPARKNUMBER,VEHICLETYPE,SESSIONSTART,SESSIONEND,TOTAL_CHARGE,DURATION,EFFECTIVE_CHARGE,LOT_NUMBER,LABEL_TYPE
4404,0PL81-L-00069-20211025013529,PL81,C,25/10/2021 1:35,25/10/2021 1:36,0.00,1,0.00,69,1
1979,0PL81-E-00001-20211014214721,PL81,C,14/10/2021 21:47,15/10/2021 19:01,0.00,1274,0.00,186,1
1887,0PL81-E-00001-20211014190901,PL81,C,14/10/2021 19:09,15/10/2021 7:12,0.00,723,0.00,174,1
2745,0PL81-E-00001-20211018205342,PL81,C,18/10/2021 20:53,18/10/2021 22:54,0.00,121,0.00,286,1
9508,0PL81-E-00001-20211008215735,PL81,C,8/10/2021 21:57,8/10/2021 22:00,0.00,3,0.00,133,0
...,...,...,...,...,...,...,...,...,...,...
7068,0PL81-E-00001-20210824205011,PL81,C,24/8/2021 20:50,24/8/2021 23:41,3.42,171,3.42,180,0
7891,0PL81-E-00001-20210909203048,PL81,C,9/9/2021 20:30,9/10/2021 0:43,5.06,253,5.06,333,0
814,0PL81-E-00001-20211008211534,PL81,C,8/10/2021 21:15,9/10/2021 12:01,0.00,886,0.00,215,1
6641,0PL81-E-00001-20210815023903,PL81,C,15/8/2021 2:39,15/8/2021 3:16,0.74,37,0.74,150,0


In [6]:
#y is the output and X is the input to train
y = currentdf['LABEL_TYPE']
X = currentdf.drop(['SESSIONID','CARPARKNUMBER','LABEL_TYPE','LOT_NUMBER'], axis = 1)
X.head()

,VEHICLETYPE,SESSIONSTART,SESSIONEND,TOTAL_CHARGE,DURATION,EFFECTIVE_CHARGE
4404,C,25/10/2021 1:35,25/10/2021 1:36,0.0,1,0.0
1979,C,14/10/2021 21:47,15/10/2021 19:01,0.0,1274,0.0
1887,C,14/10/2021 19:09,15/10/2021 7:12,0.0,723,0.0
2745,C,18/10/2021 20:53,18/10/2021 22:54,0.0,121,0.0
9508,C,8/10/2021 21:57,8/10/2021 22:00,0.0,3,0.0


In [7]:
#Used LabelEncoder to get numeric values
le = LabelEncoder()
X['VEHICLETYPE'] = le.fit_transform(X['VEHICLETYPE'])

X['SESSIONSTART'] = pd.to_datetime(X['SESSIONSTART'], format='%d/%m/%Y %H:%M', infer_datetime_format=True)
X['SESSIONEND'] = pd.to_datetime(X['SESSIONEND'], format='%d/%m/%Y %H:%M', infer_datetime_format=True)

X['SESSIONSTART'] = pd.to_datetime(X['SESSIONSTART']).map(pd.Timestamp.timestamp)
X['SESSIONEND'] = pd.to_datetime(X['SESSIONEND']).map(pd.Timestamp.timestamp)

X['SESSIONSTART'] = X['SESSIONSTART'] / 60
X['SESSIONEND'] = X['SESSIONEND'] / 60

print(X.head(10))
print(y.head(10))

#Standardized
sc = StandardScaler()
X = sc.fit_transform(X)

#Return back to dataFrame
X = pd.DataFrame(X)

       VEHICLETYPE  SESSIONSTART  SESSIONEND  TOTAL_CHARGE  DURATION  \
4404             0    27252095.0  27252096.0          0.00         1   
1979             0    27237467.0  27238741.0          0.00      1274   
1887             0    27237309.0  27238032.0          0.00       723   
2745             0    27243173.0  27243294.0          0.00       121   
9508             0    27228837.0  27228840.0          0.00         3   
2568             0    27241775.0  27242358.0          0.00       583   
9434             0    27227803.0  27230273.0         26.74      2470   
7797             0    27184110.0  27184191.0          1.62        81   
11375            0    27260387.0  27260483.0          1.92        96   
11325            0    27259620.0  27259840.0          4.40       220   

       EFFECTIVE_CHARGE  
4404               0.00  
1979               0.00  
1887               0.00  
2745               0.00  
9508               0.00  
2568               0.00  
9434               0.00  

In [8]:
#Tuning with Hyper Parameters
from sklearn.metrics import accuracy_score
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
nb_classifier = GaussianNB()

nb_classifier.fit(X_train, y_train)
y_pred_test_tuned = nb_classifier.predict(X_test)   
accuracy_score(y_test, y_pred_test_tuned)

0.9186147186147187

In [9]:
import pickle
# create an iterator object with write permission - model.pkl
with open('NaiveBayes_pkl_Latest2.pkl', 'wb') as files:
    pickle.dump(nb_classifier, files)
    
model = pickle.load(open('NaiveBayes_pkl_Latest2.pkl', 'rb'))
model.score(X_test, y_test)

0.9186147186147187